# Pre-processing
### Cleaning
We start by using a regex to remove any HTML elements, to clean the text data.

### Lowercase
We then lowercase every word to ensure uniformity and prevent the model from treating words differently based on their capitalization.

### Special Characters / Punctuation
For this particular problem, special characters and punctuation marks don't contribute much to the meaning of the text and can be removed (opinion could be different if this was a sentiment analysis instead of a topic classification problem)

### Tokenization
We then split the text into individual words/tokens, as it helps capture semantic more effectively.

### Stopwords
Stopwords are common words that occur frequently in language, but, in a topic classification problem, carry virtually no useful information. Removing them reduces the noise in the data.

### Stemming
We reduce words to their base or root form, as it reduces the size of vocabulary and the dimensionality of the feature space.

In [22]:
import pandas as pd


test_data = pd.read_csv('training_data.csv', delimiter=',', quotechar='"')

# Cleaning
test_data['text'] = test_data['text'].str.replace(r'#\d+;', '', regex=True)

# Lowercase
test_data['text'] = test_data['text'].str.lower()

# Special Characters / Punctuation

test_data['text'] = test_data['text'].str.replace(r'[^\w\s]', '', regex=True)

# Tokenization
import nltk
test_data['tokens'] = test_data['text'].apply(nltk.word_tokenize)

# Stopwords
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
test_data['tokens'] = test_data['tokens'].apply(lambda x: [word for word in x if word.lower() not in stop_words])

# Stemming
from nltk import PorterStemmer
stemmer = PorterStemmer()

def stem_tokens(tokens):
    return [stemmer.stem(token) for token in tokens]

test_data['tokens'] = test_data['tokens'].apply(stem_tokens)

print(test_data['tokens'])



0         [wall, st, bear, claw, back, black, reuter, re...
1         [carlyl, look, toward, commerci, aerospac, reu...
2         [oil, economi, cloud, stock, outlook, reuter, ...
3         [iraq, halt, oil, export, main, southern, pipe...
4         [oil, price, soar, alltim, record, pose, new, ...
                                ...                        
119995    [pakistan, musharraf, say, wont, quit, armi, c...
119996    [renteria, sign, topshelf, deal, red, sox, gen...
119997    [saban, go, dolphin, yet, miami, dolphin, put,...
119998    [today, nfl, game, pittsburgh, ny, giant, time...
119999    [net, get, carter, raptor, indianapoli, allsta...
Name: tokens, Length: 120000, dtype: object


1. Should we remove numbers?
2. Negation words


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

# Assuming you have already preprocessed the data and created feature matrices X_train and X_test
X_train = news_data['text']
y_train = news_data['label']
X_val = test_data['text']
y_val = test_data['label']
# Initialize CountVectorizer to convert text data into numerical features
vectorizer = CountVectorizer(analyzer='word',max_features=5000,lowercase=True,stop_words='english',ngram_range=(1,2))
# Fit the vectorizer on training data and transform the training data
X_train = vectorizer.fit_transform(X_train)
# Transform the test data using the fitted vectorizer
X_test = vectorizer.transform(X_val)


In [8]:
print(vectorizer.vocabulary_)

{'wall': 4794, 'st': 4210, 'bears': 518, 'black': 574, 'reuters': 3727, 'short': 4045, 'street': 4286, 'band': 483, 'seeing': 3950, 'green': 1885, 'reuters reuters': 3735, 'wall street': 4795, 'looks': 2547, 'commercial': 964, 'aerospace': 192, 'private': 3395, 'investment': 2239, 'firm': 1678, 'group': 1894, 'making': 2605, 'controversial': 1048, 'plays': 3277, 'defense': 1202, 'industry': 2169, 'quietly': 3520, 'placed': 3256, 'market': 2640, 'oil': 3024, 'economy': 1395, 'stocks': 4263, 'outlook': 3091, 'soaring': 4119, 'crude': 1116, 'prices': 3383, 'plus': 3290, 'worries': 4940, 'earnings': 1375, 'expected': 1528, 'stock': 4259, 'week': 4846, 'summer': 4347, 'stock market': 4261, 'iraq': 2255, 'exports': 1544, 'main': 2595, 'southern': 4157, 'pipeline': 3245, 'authorities': 449, 'export': 1542, 'intelligence': 2209, 'showed': 4053, 'rebel': 3591, 'militia': 2742, 'strike': 4292, 'infrastructure': 2179, 'official': 3015, 'said': 3839, 'saturday': 3872, 'official said': 3016, 'said 

In [4]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
model_nb = MultinomialNB()
model_dt = DecisionTreeClassifier(random_state=123)
model_rf = RandomForestClassifier(random_state=123)
model_lr = LogisticRegression(solver='lbfgs', max_iter=1000)

In [5]:
model_nb = model_nb.fit(X=X_train,y=y_train)
model_dt = model_dt.fit(X=X_train,y=y_train)
model_rf = model_rf.fit(X=X_train,y=y_train)
model_lr = model_lr.fit(X=X_train,y=y_train)

In [36]:
y_pred_nb = model_nb.predict(X_val)
y_pred_dt = model_dt.predict(X_val)
y_pred_rf = model_rf.predict(X_val)
y_pred_lr = model_lr.predict(X_val)

ValueError: could not convert string to float: 'Three quarters of financial firms still conducting anti-money laundering checks manually'

In [32]:
from sklearn.metrics import accuracy_score, confusion_matrix
print("Naive Bayes", accuracy_score(y_val,y_pred_nb))
print(confusion_matrix(y_val,y_pred_nb))
print()
print("Decision Tree", accuracy_score(y_val,y_pred_dt))
print(confusion_matrix(y_val,y_pred_dt))
print()
print("Random Forest", accuracy_score(y_val,y_pred_rf))
print(confusion_matrix(y_val,y_pred_rf))
print()
print("Logistic Regression", accuracy_score(y_val,y_pred_lr))
print(confusion_matrix(y_val,y_pred_lr))

Naive Bayes 70.0 %
[[4 0 1 0]
 [0 4 1 0]
 [1 0 4 0]
 [1 1 1 2]]

